<a href="https://colab.research.google.com/github/sudeepmenon17/PytorchforNN/blob/main/pytorch_to_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install google-auth
from google.colab import auth
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
  # !pip install torchvision
  # !pip install onnx
  # !pip install tensorflow
  # !pip install onnx_tf
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.functional as F
import onnx
from onnx_tf.backend import prepare

In [12]:
# Create a Model class
class Model(nn.Module):
  def __init__(self,in_features=6, h1 = 61, h2 = 60, h3=40, out_features=1):
    super().__init__()
    self.fc1 = nn.Linear(in_features,h1)
    self.dropout1 = nn.Dropout(0.05)
    self.fc2 = nn.Linear(h1,h2)
    self.dropout2 = nn.Dropout(0.05)
    self.fc3 = nn.Linear(h2,h3)
    # self.dropout3 = nn.Dropout(0.25)
    self.out = nn.Linear(h3,out_features)

  def forward(self,x):
    x = F.sigmoid(self.fc1(x))
    # x = self.dropout1(x)
    x = F.sigmoid(self.fc2(x))
    # x = self.dropout2(x)
    x = F.sigmoid(self.fc3(x))
    # x = self.dropout3(x)
    x = self.out(x)

    return x

In [13]:
import torch
import torch.onnx
import torchvision.models as models
import onnx
from onnx_tf.backend import prepare

# Step 1: Export PyTorch model to ONNX format
def export_pytorch_to_onnx(model, dummy_input, onnx_file_path):
    torch.onnx.export(model, dummy_input, onnx_file_path, export_params=True, opset_version=11)

# Step 2: Convert ONNX model to TensorFlow SavedModel format
def convert_onnx_to_tf(onnx_file_path, tf_export_path):
    onnx_model = onnx.load(onnx_file_path)
    tf_rep = prepare(onnx_model)
    tf_rep.export_graph(tf_export_path)


# Load the Saved Pytorch Model
model = Model()
model.load_state_dict(torch.load('/content/drive/MyDrive/airfoil_model.pt'))
model.eval()

# Create a dummy input tensor with the same shape as your input data
dummy_input = torch.randn(1,6)  # Adjust shape as necessary

# Define file paths
onnx_file_path = "/content/drive/MyDrive/airfoil_model.onnx"
tf_export_path = "/content/drive/MyDrive/airfoil_model_tf"

# Perform the conversion
export_pytorch_to_onnx(model, dummy_input, onnx_file_path)
convert_onnx_to_tf(onnx_file_path, tf_export_path)

print(f"Model successfully converted to TensorFlow SavedModel format and saved in {tf_export_path} directory")


KeyError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/onnx_tf/backend_tf_module.py", line 99, in __call__  *
        output_ops = self.backend._onnx_node_to_tensorflow_op(onnx_node,
    File "/usr/local/lib/python3.10/dist-packages/onnx_tf/backend.py", line 347, in _onnx_node_to_tensorflow_op  *
        return handler.handle(node, tensor_dict=tensor_dict, strict=strict)
    File "/usr/local/lib/python3.10/dist-packages/onnx_tf/handlers/handler.py", line 58, in handle  *
        cls.args_check(node, **kwargs)
    File "/usr/local/lib/python3.10/dist-packages/onnx_tf/handlers/backend/gemm.py", line 28, in args_check  *
        x = kwargs["tensor_dict"][node.inputs[0]]

    KeyError: 'onnx::Gemm_0'
